In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import NuSVR, LinearSVR
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [10]:
train_data_path = './dataset/train_data.csv'
train_data = pd.read_csv(train_data_path)
train_data.drop(columns=['Name'], inplace=True)
X = train_data.to_numpy(dtype='float64')
X.shape
y = X[:, -1]
X = X[:, :-1]
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10)


(2278, 9)
(2278,)


## Linear Regression

In [11]:
LR = LinearRegression()
LR.fit(X_train, y_train)
prediction = LR.predict(X_test)
error = prediction - y_test
MSE = np.sum(np.power(error, 2)) / error.shape[0]
print(MSE)

0.06725315553980178


## Neural Network

In [16]:
MLP = MLPRegressor(hidden_layer_sizes=200, batch_size=100, verbose=True, learning_rate='adaptive', learning_rate_init=1e-7, max_iter=5000)
MLP.fit(X_train, y_train)
prediction = MLP.predict(X_test)
error = prediction - y_test
MSE = np.sum(np.power(error, 2)) / error.shape[0]
print(MSE)

Iteration 1, loss = 7.08125394
Iteration 2, loss = 6.90302043
Iteration 3, loss = 6.73481644
Iteration 4, loss = 6.57620235
Iteration 5, loss = 6.42737245
Iteration 6, loss = 6.28883595
Iteration 7, loss = 6.15518990
Iteration 8, loss = 6.03531901
Iteration 9, loss = 5.92099899
Iteration 10, loss = 5.81528446
Iteration 11, loss = 5.71677163
Iteration 12, loss = 5.62602865
Iteration 13, loss = 5.53853982
Iteration 14, loss = 5.46146604
Iteration 15, loss = 5.38936260
Iteration 16, loss = 5.32242856
Iteration 17, loss = 5.26331097
Iteration 18, loss = 5.20589902
Iteration 19, loss = 5.15337402
Iteration 20, loss = 5.10693422
Iteration 21, loss = 5.06430259
Iteration 22, loss = 5.02400012
Iteration 23, loss = 4.98823622
Iteration 24, loss = 4.95567053
Iteration 25, loss = 4.92601365
Iteration 26, loss = 4.90011710
Iteration 27, loss = 4.87583363
Iteration 28, loss = 4.85486709
Iteration 29, loss = 4.83525197
Iteration 30, loss = 4.81824826
Iteration 31, loss = 4.80236970
Iteration 32, los

## SVM

In [26]:
NSVR = NuSVR(verbose=True, kernel='poly', degree=5, C=5)
NSVR.fit(X_train, y_train)
prediction = NSVR.predict(X_test)
error = prediction - y_test
MSE = np.sum(np.power(error, 2)) / error.shape[0]
print(MSE)


[LibSVM]0.056154979761821405


In [22]:
LSVR = LinearSVR(verbose=False, C=8)
LSVR.fit(X_train, y_train)
prediction = LSVR.predict(X_test)
error = prediction - y_test
MSE = np.sum(np.power(error, 2)) / error.shape[0]
print(MSE)

0.10910438925207887


d:\anaconda3\envs\ai-final\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\anaconda3\envs\ai-final\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Principal Component Analysis (PCA)

In [27]:
PCAmodel = PCA()
PCAmodel.fit(X)
PCAmodel.score(X)

-25.181448716675828